# Анализ и парсинг

Импорты


In [8]:
import os
import networkx as nx
import plotly.graph_objects as go
import pandas as pd

Спарсим директории

In [9]:

def build_directory_tree(path):

    if not os.path.isdir(path):
        return path  # или можно вернуть None/выбросить исключение
    
    tree = {}
    for entry in os.listdir(path):
        full_path = os.path.join(path, entry)
        if os.path.isdir(full_path):
            tree[entry] = build_directory_tree(full_path)
        else:
            tree[entry] = 'file'
    
    return tree
repo_tree = build_directory_tree('../data/test_repos/')

Первая версия визуализации(на уровне дерева файлов)

In [14]:

def dict_to_graph(data, parent=None, G=None):
    if G is None:
        G = nx.DiGraph()
    for key, value in data.items():
        G.add_node(key)
        if parent:
            G.add_edge(parent, key)
        if isinstance(value, dict):
            dict_to_graph(value, key, G)
    return G

def visualize_graph(G):
    pos = nx.spring_layout(G, seed=42)  # Можно заменить на nx.kamada_kawai_layout или graphviz

    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x += [x0, x1, None]
        edge_y += [y0, y1, None]

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='gray'),
        hoverinfo='none',
        mode='lines'
    )

    node_x, node_y, node_text = [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(str(node))

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="bottom center",
        hoverinfo='text',
        marker=dict(
            size=30,
            color='lightblue',
            line=dict(width=2, color='darkblue')
        )
    )

    fig = go.Figure(data=[edge_trace, node_trace],
        layout=go.Layout(
            title='Граф иерархии словаря',
            showlegend=False,
            hovermode='closest',
            margin=dict(b=20, l=20, r=20, t=50),
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
        )
    )
    fig.show()



G = dict_to_graph(repo_tree)
visualize_graph(G)


In [16]:
from langchain_openai.chat_models import ChatOpenAI


llm = ChatOpenAI(
    model="gpt://" + os.environ["folder_id"] + "/yandexgpt/rc",
    api_key=os.environ["api_key"],
    base_url="https://llm.api.cloud.yandex.net/v1"
)
llm.invoke('мяу')

KeyError: 'folder_id'